# Detect DICOM Image PHI and Redact PHI
- Recommended cluster: >= 15.4 LTS ML (avoid Serverless)

## Setup `pixels` package (if you haven't done so)
1. [Create a git folder](https://docs.databricks.com/aws/en/repos/git-operations-with-repos) cloning the [pixels](https://github.com/databricks-industry-solutions/pixels) package
2. Then run the [`config/setup.py`]($./config/setup) script in the repo folder

In [0]:
%run ./config/setup

Installing Pixels Solution Accelerator dependencies from  /Workspace/Users/yen.low@databricks.com/pixel/pixels
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
petastorm 0.12.1 requires pyspark>=2.1.0, which is not installed.
boto3 1.34.39 requires botocore<1.35.0,>=1.34.39, but you have botocore 1.27.59 which is incompatible.
databricks-feature-engineering 0.8.0 requires mlflow-skinny[databricks]<3,>=2.11.0, but you have mlflow-skinny 3.1.0 which is incompatible.
datasets 2.19.1 requires fsspec[http]<=2024.3.1,>=2023.1.0, but you have fsspec 2022.10.0 which is incompatible.
huggingface-hub 0.27.1 requires fsspec>=2023.5.0, but you have fsspec 2022.10.0 which is incompatible.
jupyter-server 1.23.4 requires anyio<4,>=3.1.0, but you have anyio 4.9.0 which is incompatible.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 1.26.4 which is incompatible.
s3t

'Done'

Installing Pixels Solution Accelerator dependencies from  /Workspace/Users/yen.low@databricks.com/pixel/pixels
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
petastorm 0.12.1 requires pyspark>=2.1.0, which is not installed.
boto3 1.34.39 requires botocore<1.35.0,>=1.34.39, but you have botocore 1.27.59 which is incompatible.
databricks-feature-engineering 0.8.0 requires mlflow-skinny[databricks]<3,>=2.11.0, but you have mlflow-skinny 3.1.1 which is incompatible.
datasets 2.19.1 requires fsspec[http]<=2024.3.1,>=2023.1.0, but you have fsspec 2022.10.0 which is incompatible.
huggingface-hub 0.27.1 requires fsspec>=2023.5.0, but you have fsspec 2022.10.0 which is incompatible.
jupyter-server 1.23.4 requires anyio<4,>=3.1.0, but you have anyio 4.9.0 which is incompatible.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 1.26.4 which is incompatible.
s3t

'Done'

Installing Pixels Solution Accelerator dependencies from  /Workspace/Users/yen.low@databricks.com/pixel/pixels
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
petastorm 0.12.1 requires pyspark>=2.1.0, which is not installed.
boto3 1.34.39 requires botocore<1.35.0,>=1.34.39, but you have botocore 1.27.59 which is incompatible.
databricks-feature-engineering 0.8.0 requires mlflow-skinny[databricks]<3,>=2.11.0, but you have mlflow-skinny 3.1.2 which is incompatible.
datasets 2.19.1 requires fsspec[http]<=2024.3.1,>=2023.1.0, but you have fsspec 2022.10.0 which is incompatible.
huggingface-hub 0.27.1 requires fsspec>=2023.5.0, but you have fsspec 2022.10.0 which is incompatible.
jupyter-server 1.23.4 requires anyio<4,>=3.1.0, but you have anyio 4.9.0 which is incompatible.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 1.26.4 which is incompatible.
s3t

'Done'

In [0]:
%pip install -U pydicom

  Obtaining dependency information for pydicom from https://files.pythonhosted.org/packages/27/a6/98651e752a49f341aa99aa3f6c8ba361728dfc064242884355419df63669/pydicom-3.0.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.4 MB ? eta -:--:--
   ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.2/2.4 MB 5.1 MB/s eta 0:00:01
   ━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.5/2.4 MB 6.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.8/2.4 MB 8.0 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━ 1.3/2.4 MB 9.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━ 1.8/2.4 MB 10.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 2.4/2.4 MB 11.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 11.1 MB/s eta 0:00:00
  Attempting uninstall: pydicom
    Found existing installation: pydicom 2.4.4
    Uninstalling pydicom-2.4.4:
      Successfully uninstalled pydicom-2.4.4
Note: you may need to restart the kernel 

In [0]:
import pandas as pd
from dbx.pixels.logging import LoggerProvider

logger = LoggerProvider()

In [0]:
dbutils.widgets.text("table", "hls_radiology.tcia.midi_b_val_subset", label="table of DICOM paths")
table = dbutils.widgets.get("table")
dbutils.widgets.text("output_dir", "/Volumes/hls_radiology/tcia/redacted", label="volumes folder for storing redacted images")
output_dir = dbutils.widgets.get("output_dir")

### Load input dataframe
This assumes a table of DICOM paths already exists with the UC path set in `table`.<br>
To ingest DICOM files into a delta spark table, see [01-dcm-demo]($./01-dcm-demo) for examples.

`VLMPhiExtractor` requires that input be must be ONE of the following:
1. a .dcm file path (e.g. `/Volumes/<catalog>/<schema>/2.1.656.0.2.8048482.9.537.165816238/1-1.dcm`)
2. image file path (e.g. `/Volumes/<catalog>/<schema>/2.1.656.0.2.8048482.9.537.165816238/1-1.jpg`)
3. image encoded as a base64 string required by VLM (e.g. `/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwc...`)

In [0]:
df = spark.table(table)
display(df)

SeriesInstanceUID PatientID BodyPartExamined Collection CollectionURI DateReleased FileSize ImageCount Manufacturer ManufacturerModelName Modality ProtocolName SeriesDate SeriesDescription SeriesNumber SoftwareVersions StudyDate StudyDesc StudyInstanceUID ThirdPartyAnalysis TimeStamp PatientID_masked series_id_masked path_masked pixel_hash_masked path series_id pixel_hash frame is_identical jpg_base64str jpg_base64str_masked is_different has_phi 2.1.656.0.2.8048482.9.537.1658163530109825238 1273299863 CHEST MIDI-B-Synthetic-Validation https://doi.org/10.7937/cf2p-aw56 2025-05-02 00:00:00.0 15283394 1 null null DX null 2017-08-11 00:00:00.0 AP 1 1.4 2017-08-11 00:00:00.0 XR CHEST 1 VIEW AP 2.1.656.0.2.8048482.9.537.1460950126025919742 NO 2025-02-23 23:30:04.0 MIDI_1_1_129 1.3.6.1.4.1.14519.5.2.1.8700.9920.153121710989564115753937377722 dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/1.3.6.1.4.1.14519.5.2.1.8700.9920.153121710989564115753937377722/1-1.dcm 2769535f16e033e3878881a0949b2be16683959e dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/2.1.656.0.2.8048482.9.537.1658163530109825238/1-1.dcm 2.1.656.0.2.8048482.9.537.1658163530109825238 f1fa359d9c6bdf20019a2cd9a006fd788234164a 1-1 null ... ... true 0 2.4.357.1.0.9259302.8.497.3349013607548370215 1523933739 BREAST MIDI-B-Synthetic-Validation https://doi.org/10.7937/cf2p-aw56 2025-05-02 00:00:00.0 10634156 77 GE MEDICAL SYSTEMS SIGNA EXCITE MR Ordered for Joseph Berry by Dr. Hoover 2014-11-10 00:00:00.0 SAG T2 (FAT-SAT) LEFT for Joseph Berry 20141110 11 2014-11-10 00:00:00.0 MR BREAST, BILATERAL W/WO CONT at HGAWC 2.4.357.1.0.9259302.8.497.2891140086790562459 NO 2025-02-23 23:43:43.0 MIDI_1_1_132 1.3.6.1.4.1.14519.5.2.1.8700.9920.328655332873086989977769633175 dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/1.3.6.1.4.1.14519.5.2.1.8700.9920.328655332873086989977769633175/1-70.dcm e3e49dadc0a829afac70a5e1e7a7f07f3d9e59b5 dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/2.4.357.1.0.9259302.8.497.3349013607548370215/1-70.dcm 2.4.357.1.0.9259302.8.497.3349013607548370215 e3e49dadc0a829afac70a5e1e7a7f07f3d9e59b5 1-70 null /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAEAAQABAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APHjaxKAdgAOCQWOMUiwQLF80Z8w5I5IHXH9KBbIqlyuFIwpzyee1NECBwmz23A8fjSlIlVvlY55B9PrT1t0KnIZjjrngH39qUokbrxuAGOv44FM8kPExBVec9cYHWhYyx2jBIGcjn+dPjRVJHJJ44PQUgbEgYgDPyg54H4UYInLK7EjHBGcntTEIU7SScHtnn/P9KczjGxVBOcqT2FI5CPs5yDjjvS/ffKYLLyRTjH8zMFIXlsg9KbjPH3iR0K/ShMMAGbLDjGRkj0/CkWMKrEtxjC59aXytpChyWxyR2pMGQMZWB5wPWneYjQMeBz2/wA+uaJI1iPzZOeG9jQpZthKgZ5x3PrTiSsh8znuQvAP+eK6vwhCrWd8FEm92XGw7goHU49KNRa3d5fPkEexWXnnc2OMCubYmWMScsSBkEYGcentUGzzO7buvAzk02OMcrkg5x97gH1pT5cilVQLjlufb/61GNyAMcYA3Cm5RT0LKBnk+9SsxEaBjnByVxxik2qsm5cbwM5PQ/X6Z/SmgsM7yCNoJA6E0AFlH8SAFfl9DzQ5O9QAQMBiSOcGkx8+VAyPbkU//VMMZBx83XnnvSYI+bG4ljz1A5/lyKaB9wMDtbpgH1NMfI2/KAR2PWgkgHlmXAxtPT+tO3uJEK4XYcgY5z6GnLFvy4YAq2WBGMc5/wAKTI3sNqHpn6f1pDnIBA8vOQB1FEu9mbAXcO4OefrTl2BcscqMj3X8fSo02kFg2OR1POKUjG3byWHOaeEJyxDLgcEHIOKaWAYkAhc/L78dTTmD4DMqu3XJ7Z7mgIPNILZUDrjOKnYqgGCJHfOQRyPT6cZrsPBdu/8AZ95NEjlA3zeWQCMep9O9V9ai33WPKjZHJb5B/WuYupI3kPlptI5yx+77fWqqvuf5fvE7sA4BNOOFdmIAkOG2DjDZ6f1pUVtzFl3cEjHY+tMjcqq7kznnr1B4xUkg65U5P8I7+hp0rIQQWYMT1656mopCu9mzk8YWlUKqjrnOQfQ0bt7BAGIPB6daJTGpPQquBg+3+TTTukdR823g5x096cxTftByPukHuc00hkZlkBZeij+lKAGJC54YcZx+FBQo7Eu/YcDnHWkwFZg4IAB4Hb0/r+VPIykZ4G4ZO7nmoyQyA4xjtnrj1707AkAIO1x19PakDPGVAbBxzk5GOKDsGQFGRlQc4/yaG2FgkYZVOPmPf2zQUXJz9727kUocqE2kkA5zjP8AkUjSKJF64zyelKEJUt0OCSp7jtTmlIUqvzL/ALXb/OKNjqGJyzEcsOw9fwpQJDcbi2C3zZOMZ6Zr0nwdBDF4TuJpLjEbykMGXBI4+79MVzev3yTXDMx4IGB0HTrx3rn5zloVZUO4Ek564/l0qsoX5h8pYHOB0H1qUHOMY6cFece1RhiqMSBk9dp6Aio4GzFk7izentU+9cxkb

In [0]:
in_df = df.select("path", "has_phi")
in_df = in_df.repartition(8)

## Use Spark ML Pipeline to build a PHI detection and redaction workflow
### Pipeline 1: Without a filter (VLMPhiDetector + OcrRedactor)

Build an end-to-end `pyspark.ml.Pipeline` defined with `VLMPhiDetector` and `OcrRedactor` as `Transformer` stages in a workflow. This utilizes Spark ML for highly efficient parallel processing.

1. `VLMPhiDetector` detects and extracts PHI entities using a VLM (set by `endpoint` parameter)
2. `OcrRedactor` masks PHI from images using EasyOCR to detect text boundaries

```
# As a pipeline
Pipeline(stages=[VLMPhiDetector("databricks-claude-3-7-sonnet"), 
                OcrRedactor()])
```
This is wrapped in `DicomPhiPipeline` where you provide the inputs to `VLMPhiDetector` and `OcrRedactor`. Here, `redact_even_if_undetected=False` so `OcrRedactor` processes all the .dcm whether or not they have PHI detected by the prior `VLMPhiDetector`

In [0]:
from dbx.pixels.dicom.dicom_phi import DicomPhiPipeline

pipeline = DicomPhiPipeline(endpoint="databricks-claude-3-7-sonnet",
                            output_dir=output_dir,
                            redact_even_if_undetected=True,
                            inputCol="path",
                            detectCol="response",
                            outputCol="path_redacted")
model_redact_undetected = pipeline.fit(in_df)
out_df = model_redact_undetected.transform(in_df)
display(out_df)

path,has_phi,response,path_redacted
dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/2.3.957.1.1.1204841.4.221.1244497244821555782/1-1.dcm,1,"List(List(HAYES JESSICA, 01.03.2012, 12.15.1986), 21, 962, 983, null)",/Volumes/hls_radiology/tcia/redacted/2.3.957.1.1.1204841.4.221.1244497244821555782_1-1.dcm
dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/3.4.868.1.1.3410519.2.594.1727310115838398258/1-2.dcm,0,"List(List(), 6, 906, 912, null)",/Volumes/hls_radiology/tcia/redacted/3.4.868.1.1.3410519.2.594.1727310115838398258_1-2.dcm
dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/2.4.332.0.2.8029662.9.504.3152287158250332395/1-314.dcm,0,"List(List(), 6, 511, 517, null)",null
dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/1.2.692.1.2.8881663.7.630.2602716578066938264/1-0186.dcm,0,"List(List(), 6, 175, 181, null)",null
dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/3.1.874.1.3.8955936.3.406.2571635843887543751/1-1155.dcm,0,"List(List(), 6, 186, 192, null)",null
dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/3.3.887.0.0.7990059.2.604.6609992527253610405/1-02.dcm,0,"List(List(), 6, 175, 181, null)",null
dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/3.3.643.1.2.3791796.5.188.3075860933482453746/1-1.dcm,0,"List(List(), 6, 794, 800, null)",/Volumes/hls_radiology/tcia/redacted/3.3.643.1.2.3791796.5.188.3075860933482453746_1-1.dcm
dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/3.2.963.0.1.1744807.5.560.2925915423073421345/1-48.dcm,0,"List(List(), 6, 511, 517, null)",null
dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/2.1.239.0.1.5573651.5.438.2234045507897739313/1-1.dcm,1,"List(List(JOHNSON LARRY [M], 03.09.2019, 01.30.1968), 24, 934, 958, null)",/Volumes/hls_radiology/tcia/redacted/2.1.239.0.1.5573651.5.438.2234045507897739313_1-1.dcm
dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/1.3.699.1.2.1566659.8.694.3677882864699772127/1-1.dcm,0,"List(List(), 6, 962, 968, null)",/Volumes/hls_radiology/tcia/redacted/1.3.699.1.2.1566659.8.694.3677882864699772127_1-1.dcm


## Evaluate against ground truth

In [0]:
from pyspark.sql.functions import split, col, when, size, isnotnull
import pandas as pd

extracted_df = (out_df
    .withColumn("phi_detected", when(size(col("response.content"))>1, True).otherwise(False))
    .withColumn("redacted", isnotnull(col("path_redacted")))
    .select("has_phi", "phi_detected", "redacted")
)
display(extracted_df)

has_phi,phi_detected,redacted
1,true,true
0,false,true
0,false,false
0,false,false
0,false,false
0,false,false
0,false,true
0,false,false
1,true,true
0,false,true


In [0]:
from dbx.pixels.dicom.dicom_utils import get_classifer_metrics

# Performance of VLMPhiDetector
get_classifer_metrics(extracted_df, col_pred="phi_detected")

{'precision': 1.0,
 'specificity': 1.0,
 'npv': 1.0,
 'recall': 1.0,
 'f1': 1.0,
 'accuracy': 1.0}

In [0]:
# Performance of OcrRedactor
get_classifer_metrics(extracted_df, col_pred="redacted")

{'precision': 0.2857142857142857,
 'specificity': 0.5833333333333334,
 'npv': 1.0,
 'recall': 1.0,
 'f1': 0.4444444444444445,
 'accuracy': 0.6428571428571429}

## Interpreting the performance metrics
The metrics are defined as follows:
- **Recall** measures proportion of positive labels (i.e. has PHI) correctly predicted
- **Specificity** measures proportion of negative labels (i.e. non-PHI) correctly predicted
- **Precision** measures of those predicted to be positive, how many actually have PHI. Its difference from 1 represents the false discovery rate
- **Negative Predictive Value (NPV)** measures of those predicted to be negative, how many actually are non-PHI. Its difference from 1 represents the false omission rate.
- **F1** is the harmonic mean of Precision and Recall `F1 = 2*(precision * recall)/(precision + recall)`
- **Accuracy** is the average of Recall and Specificity weighted by proportion of positive and negative labels respectively 

As these metrics are all related and trade off against one another. It is important to determine whether it is more critical to not miss any PHI (i.e. high recall) and risk overredacting non-PHI or if it is acceptable to risk missing PHI (i.e. underredact) in favor of not falsely predicting non-PHI as PHI. The latter becomes more important when there is an unacceptable risk of falsely predicting a negative as positive (i.e. diagnosed with cancer when one does not have it). 

In this example, the former is preferred as the risk of missing PHI is too high. **So we want close to zero false omission rate (i.e. high NPV) and high recall which is the case for both the detector and redactor. Both have 100% NPV and 100% recall**

### Pipeline 2: With a filter (VLMPhiDetector + FilterTransformer + OcrRedactor)
As `OcrRedactor` tends to overredact (i.e. has many false positives), for best performance, we recommend adding a `FilterTransformer` that filters according to the output of `VLMPhiDetector` and is subsequently fed into `OcrRedactor`. 

1. `VLMPhiDetector` detects and extracts PHI entities using a VLM
2. `FilterTransformer` filters only the PHI rows detected by `VLMPhiDetector`
3. `OcrRedactor` works on the filtered PHI rows and masks PHI from images using EasyOCR
```
# As a pipeline
Pipeline(stages=[VLMPhiDetector("databricks-claude-3-7-sonnet"), 
                FilterTransformer(),
                OcrRedactor()])
```

This doesn't seem to affect the redactor's ability to miss PHI (i.e. NPV remains perfect at 1).

In [0]:
pipeline_redact_only_detected = DicomPhiPipeline(endpoint="databricks-claude-3-7-sonnet",
                                                 output_dir=output_dir,
                                                 redact_even_if_undetected=False)
model_redact_only_detected = pipeline_redact_only_detected.fit(in_df)
out_df_redact_only_detected = model_redact_only_detected.transform(in_df)
display(out_df_redact_only_detected)

path,has_phi,response,filtered,path_redacted
dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/2.3.957.1.1.1204841.4.221.1244497244821555782/1-1.dcm,1,"List(List(HAYES JESSICA, 01.03.2012, 12.15.1986), 21, 962, 983, null)",dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/2.3.957.1.1.1204841.4.221.1244497244821555782/1-1.dcm,/Volumes/hls_radiology/tcia/redacted/2.3.957.1.1.1204841.4.221.1244497244821555782_1-1.dcm
dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/3.4.868.1.1.3410519.2.594.1727310115838398258/1-2.dcm,0,"List(List(), 6, 906, 912, null)",null,null
dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/2.4.332.0.2.8029662.9.504.3152287158250332395/1-314.dcm,0,"List(List(), 6, 511, 517, null)",null,null
dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/1.2.692.1.2.8881663.7.630.2602716578066938264/1-0186.dcm,0,"List(List(), 6, 175, 181, null)",null,null
dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/3.1.874.1.3.8955936.3.406.2571635843887543751/1-1155.dcm,0,"List(List(), 6, 186, 192, null)",null,null
dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/3.3.887.0.0.7990059.2.604.6609992527253610405/1-02.dcm,0,"List(List(), 6, 175, 181, null)",null,null
dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/3.3.643.1.2.3791796.5.188.3075860933482453746/1-1.dcm,0,"List(List(), 6, 794, 800, null)",null,null
dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/3.2.963.0.1.1744807.5.560.2925915423073421345/1-48.dcm,0,"List(List(), 6, 511, 517, null)",null,null
dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/2.1.239.0.1.5573651.5.438.2234045507897739313/1-1.dcm,1,"List(List(JOHNSON LARRY [M], 03.09.2019, 01.30.1968), 24, 934, 958, null)",dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/2.1.239.0.1.5573651.5.438.2234045507897739313/1-1.dcm,/Volumes/hls_radiology/tcia/redacted/2.1.239.0.1.5573651.5.438.2234045507897739313_1-1.dcm
dbfs:/Volumes/hls_radiology/tcia/downloads/unzipped/1.3.699.1.2.1566659.8.694.3677882864699772127/1-1.dcm,0,"List(List(), 6, 962, 968, null)",null,null


In [0]:
extracted_df_redact_only_detected = (out_df_redact_only_detected
    .withColumn("phi_detected", when(size(col("response.content"))>1, True).otherwise(False))
    .withColumn("redacted", isnotnull(col("path_redacted")))
    .select("has_phi", "phi_detected", "redacted")
)
display(extracted_df_redact_only_detected)

has_phi,phi_detected,redacted
1,true,true
0,false,false
0,false,false
0,false,false
0,false,false
0,false,false
0,false,false
0,false,false
1,true,true
0,false,false


In [0]:
# Performance of VLMPhiDetector
get_classifer_metrics(extracted_df_redact_only_detected, col_pred="phi_detected")

{'precision': 1.0,
 'specificity': 1.0,
 'npv': 1.0,
 'recall': 1.0,
 'f1': 1.0,
 'accuracy': 1.0}

In [0]:
# Performance of OcrRedactor
get_classifer_metrics(extracted_df_redact_only_detected, col_pred="redacted")

{'precision': 1.0,
 'specificity': 1.0,
 'npv': 1.0,
 'recall': 1.0,
 'f1': 1.0,
 'accuracy': 1.0}

## Conclusion
As stated earlier, we emphasize high Recall (100%) and NPV (100%) which were already achieved in the first pipeline without a filter (`redact_even_if_undetected=True`). However, its Specificity (58.3%) and Precision (28.6%) were low due to overredaction of non-PHI. 

To minimize overredaction, we introduced a filter (`redact_even_if_undetected=False`) in the second pipeline such that only detected PHI will undergo redaction. This improved both the detector and redactor to 100% in all metrics.

View the files in the [output_dir](https://e2-demo-field-eng.cloud.databricks.com/explore/data/volumes/hls_radiology/tcia/redacted)